In [ ]:
import numpy as np
import pandas
import os
from urllib.request import urlretrieve
import zipfile

In [ ]:
import os
import csv
import gzip
import collections
import re
import io
import json
import xml.etree.ElementTree as ET

import requests
import pandas

In [ ]:
zip_ref = zipfile.ZipFile(filename, 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
with open('full database.xml') as xml_file:  
    tree = ET.parse(xml_file)
root = tree.getroot()

In [ ]:
ns = '{http://www.drugbank.ca}'
inchikey_template = "{ns}calculated-properties/{ns}property[{ns}kind='InChIKey']/{ns}value"
inchi_template = "{ns}calculated-properties/{ns}property[{ns}kind='InChI']/{ns}value"
SMILES_template = "{ns}calculated-properties/{ns}property[{ns}kind='SMILES']/{ns}value"
logS_template = "{ns}experimental-property/{ns}property[{ns}kind='logS']/{ns}value"
logP_template = "{ns}experimental-property/{ns}property[{ns}kind='logP']/{ns}value"
Water_Solubility_template = "{ns}experimental-property/{ns}property[{ns}kind='Water Solubility']/{ns}value"
Melting_Point_template = "{ns}experimental-property/{ns}property[{ns}kind='Melting Point']/{ns}value"
Boiling_Point_template = "{ns}experimental-property/{ns}property[{ns}kind='Boiling Point']/{ns}value"
Hydrophobicity_template = "{ns}experimental-property/{ns}property[{ns}kind='Hydrophobicity']/{ns}value"
Isoelectric_Point_template = "{ns}experimental-property/{ns}property[{ns}kind='Isoelectric Point']/{ns}value"
caco2_Permeability_template = "{ns}experimental-property/{ns}property[{ns}kind='caco2 Permeability']/{ns}value"
pKa_template = "{ns}experimental-property/{ns}property[{ns}kind='pKa']/{ns}value"
Radioactivity_template = "{ns}experimental-property/{ns}property[{ns}kind='Radioactivity']/{ns}value"

    
rows = list()
for i, drug in enumerate(root):
    row = collections.OrderedDict()
    assert drug.tag == ns + 'drug'
    row['type'] = drug.get('type')
    row['drugbank_id'] = drug.findtext(ns + "drugbank-id[@primary='true']")
    row['name'] = drug.findtext(ns + "name")
    row['description'] = drug.findtext(ns + "description")
    row['pharmacodynamics'] = drug.findtext(ns + "pharmacodynamics")
    row['mechanism-of-action'] = drug.findtext(ns + "mechanism-of-action")
    row['toxicity'] = drug.findtext(ns + "toxicity")
    row['metabolism'] = drug.findtext(ns + "metabolism")
    row['absorption'] = drug.findtext(ns + "absorption")
    row['half-life'] = drug.findtext(ns + "half-life")
    row['protein-binding'] = drug.findtext(ns + "protein-binding")
    row['route-of-elimination'] = drug.findtext(ns + "route-of-elimination")
    row['clearance'] = drug.findtext(ns + "clearance")

    row['groups'] = [group.text for group in
        drug.findall("{ns}groups/{ns}group".format(ns = ns))]
    row['targets'] = [target.text for target in
        drug.findall("{ns}targets/{ns}target".format(ns = ns))]
    row['enzymes'] = [enzyme.text for enzyme in
        drug.findall("{ns}enzymes/{ns}enzyme".format(ns = ns))]
    
    
    row['atc_codes'] = [code.get('code') for code in
        drug.findall("{ns}atc-codes/{ns}atc-code".format(ns = ns))]
    row['categories'] = [x.findtext(ns + 'category') for x in
        drug.findall("{ns}categories/{ns}category".format(ns = ns))]
    row['inchi'] = drug.findtext(inchi_template.format(ns = ns))
    row['inchikey'] = drug.findtext(inchikey_template.format(ns = ns))
    row['SMILES'] = drug.findtext(SMILES_template.format(ns = ns))
    row['logS'] = drug.findtext(logS_template.format(ns = ns))
    row['logP'] = drug.findtext(logP_template.format(ns = ns))
    row['Water_Solubility'] = drug.findtext(Water_Solubility_template.format(ns = ns))
    row['Melting_Point'] = drug.findtext(Melting_Point_template.format(ns = ns))
    row['Boiling_Point'] = drug.findtext(Boiling_Point_template.format(ns = ns))
    row['Hydrophobicity'] = drug.findtext(Hydrophobicity_template.format(ns = ns))
    row['Isoelectric_Point'] = drug.findtext(Isoelectric_Point_template.format(ns = ns))
    row['caco2_Permeability'] = drug.findtext(caco2_Permeability_template.format(ns = ns))
    row['pKa'] = drug.findtext(pKa_template.format(ns = ns))
    row['Radioactivity'] = drug.findtext(Radioactivity_template.format(ns = ns))
    
    
    # Add drug aliases
    aliases = {
        elem.text for elem in 
        drug.findall("{ns}international-brands/{ns}international-brand".format(ns = ns)) +
        drug.findall("{ns}synonyms/{ns}synonym[@language='English']".format(ns = ns)) +
        drug.findall("{ns}international-brands/{ns}international-brand".format(ns = ns)) +
        drug.findall("{ns}products/{ns}product/{ns}name".format(ns = ns))

    }
    aliases.add(row['name'])
    row['aliases'] = sorted(aliases)

    rows.append(row)

In [ ]:
print(os.listdir())

In [ ]:
def collapse_list_values(row):
    for key, value in row.items():
        if isinstance(value, list):
            row[key] = '|'.join(value)
    return row

rows = list(map(collapse_list_values, rows))

In [ ]:
#columns = ['drugbank_id', 'name', 'type', 'groups', 'atc_codes', 'categories', 
#           'inchikey', 'inchi', 'description', 'SMILES', 'logP', 'logS', 
#           'Water_Solubility']
drugbank_df = pandas.DataFrame.from_dict(rows)#[columns]
drugbank_df.head()

In [ ]:
drugbank_slim_df = drugbank_df[
    drugbank_df.groups.map(lambda x: 'approved' in x) &
    drugbank_df.inchi.map(lambda x: x is not None) &
    drugbank_df.type.map(lambda x: x == 'small molecule')
]
drugbank_slim_df.head()

In [ ]:
# write drugbank tsv
path = 'drugbank.tsv'
drugbank_df.to_csv(path, sep='\t',index=False)

# write slim drugbank tsv
path = 'drugbank-slim.tsv'
drugbank_slim_df.to_csv(path,sep='\t', index=False)

In [ ]:
# write drugbank tsv
path = 'drugbank.csv'
drugbank_df.to_csv(path, index=False)

# write slim drugbank tsv
path = 'drugbank-slim.csv'
drugbank_slim_df.to_csv(path, index=False)

## Extract protein information

In [ ]:
protein_rows = list()
for i, drug in enumerate(root):
    drugbank_id = drug.findtext(ns + "drugbank-id[@primary='true']")
    for category in ['target', 'enzyme', 'carrier', 'transporter']:
        proteins = drug.findall('{ns}{cat}s/{ns}{cat}'.format(ns=ns, cat=category))
        for protein in proteins:
            row = {'drugbank_id': drugbank_id, 'category': category}
            row['organism'] = protein.findtext('{}organism'.format(ns))
            row['known_action'] = protein.findtext('{}known-action'.format(ns))
            actions = protein.findall('{ns}actions/{ns}action'.format(ns=ns))
            row['actions'] = '|'.join(action.text for action in actions)
            uniprot_ids = [polypep.text for polypep in protein.findall(
                "{ns}polypeptide/{ns}external-identifiers/{ns}external-identifier[{ns}resource='UniProtKB']/{ns}identifier".format(ns=ns))]            
            if len(uniprot_ids) != 1:
                continue
            row['uniprot_id'] = uniprot_ids[0]
            ref_text = protein.findtext("{ns}references[@format='textile']".format(ns=ns))
            pmids = re.findall(r'pubmed/([0-9]+)', str(ref_text))
            row['pubmed_ids'] = '|'.join(pmids)
            protein_rows.append(row)

protein_df = pandas.DataFrame.from_dict(protein_rows)

In [ ]:
protein_df.head()

In [ ]:

# Read our uniprot to entrez_gene mapping
response = requests.get('http://git.dhimmel.com/uniprot/data/map/GeneID.tsv.gz', stream=True)
text = io.TextIOWrapper(gzip.GzipFile(fileobj=response.raw))
uniprot_df = pandas.read_table(text, engine='python')
uniprot_df.rename(columns={'uniprot': 'uniprot_id', 'GeneID': 'entrez_gene_id'}, inplace=True)

# merge uniprot mapping with protein_df
entrez_df = protein_df.merge(uniprot_df, how='inner')

In [ ]:
path = 'proteins.tsv'
entrez_df.to_csv(path, sep='\t', index=False)
path = 'proteins.csv'
entrez_df.to_csv(path, index=False)

In [ ]:
entrez_df.head()